In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[7]:


#! python3
# -*- encoding: utf-8 -*-
'''
@Time    :   2023/05/15 
@Author  :   Jin Yujun
@Version :   2.0
@Contact :   13738026651@163.com
'''

#测试
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

In [2]:
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pd
import numpy as np
import torch.nn as nn
from transformers import BertModel
import os

/home/jinyujun/anaconda3/envs/pytorch/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/home/jinyujun/anaconda3/envs/pytorch/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
CUDA=torch.cuda.is_available()
devices  = torch.device('cuda' if CUDA else 'cpu')

data_tensor = np.load('sentences_tokenizer_test.npy',allow_pickle=True)
for key in data_tensor.item():
    data_tensor.item()[key] = data_tensor.item()[key].to(devices)


In [4]:
class DisasterDataset(Dataset):
    def __init__(self,data):
        self.data = data.item()

    def __getitem__(self, idx):
        sentences = {}
        sentences['input_ids'] = self.data['input_ids'][idx]
        sentences['attention_mask'] = self.data['attention_mask'][idx]
        
        return sentences
    def __len__(self):
        return len(self.data['input_ids'])

class BertClassificationModel(nn.Module):
    def __init__(self,dropout,hidden_size=768):
        super(BertClassificationModel, self).__init__()
        
        model_name = 'bert-base-uncased'
        self.bert = BertModel.from_pretrained(pretrained_model_name_or_path=model_name)
        self.fc = nn.Linear(hidden_size,2)

    def forward(self, batch_sentences):   # [batch_size,1]
        
        input_ids = batch_sentences['input_ids']
        attention_mask = batch_sentences['attention_mask']
        bert_out=self.bert(input_ids=input_ids,attention_mask=attention_mask) # 模型
        last_hidden_state =bert_out[0] # [batch_size, sequence_length, hidden_size] # 变量
        bert_cls_hidden_state=last_hidden_state[:,0,:] # 变量
        fc_out=self.fc(bert_cls_hidden_state) # 模型
        return fc_out        

In [5]:
model = BertClassificationModel(0.3)
model = nn.DataParallel(model).to(devices)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

In [17]:
batchsize = 128  # 定义每次放多少个数据参加训练
    
Datas = DisasterDataset(data_tensor) # 加载训练集
test_loader = torch.utils.data.DataLoader(Datas, batch_size=batchsize, shuffle=False)#遍历train_dataloader 每次返回batch_size条数据

model.eval()  # 不启用 BatchNormalization 和 Dropout，保证BN和dropout不发生变化,主要是在测试场景下使用；
label = []
for j, data in enumerate(test_loader, 0):
    #data = data.to(devices)
    output = model(data)
    out = output.argmax(dim=1)
    out=out.tolist()
    label.extend(out)



In [25]:
label = pd.DataFrame(label)
ids = pd.read_csv('test.csv')#这里因为测试集的id不是连续的！！！所以需要读取一下
ids = ids.iloc[:,1]
label = pd.concat([ids,label],axis=1)
fieldname = ['id','target']
label.to_csv("submission.csv",index_label = fieldname)

/home/jinyujun/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


In [24]:
print(label)

      0
0     1
1     1
2     1
3     1
4     1
...  ..
3258  1
3259  1
3260  1
3261  1
3262  1

[3263 rows x 1 columns]


In [10]:
label = pd.read_csv('submission.csv')
ids = pd.read_csv('df_test.csv')

In [11]:
label = label.iloc[:,1]
ids = ids.iloc[:,1]

In [13]:
label = pd.concat([ids,label],axis=1)

In [15]:
label = label.to_csv('submission.csv',index=False)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1
